<a href="https://colab.research.google.com/github/jvishnuvardhan/TF_Lite/blob/master/34444_TFLiteConv_with_ConcreteFun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tf-nightly

     |████████████████████████████████| 521.7MB 30kB/s 
     |████████████████████████████████| 460kB 43.2MB/s 
     |████████████████████████████████| 2.9MB 46.0MB/s 


In [0]:
from time import time

import numpy as np
import tensorflow as tf
from tensorflow.lite.python.interpreter import load_delegate


def measure(size, rep, tpu):
    @tf.function(input_signature=[tf.TensorSpec([size] * 2, tf.float32)] * 2)
    def bench_func(a, b):
        x = tf.linalg.matmul(a, b)
        return tf.reduce_mean(x)
    
    def gen_input_samples():
        i = np.identity(size, np.float32)
        yield [-i, i]
        yield [i, i]

    converter = tf.lite.TFLiteConverter.from_concrete_functions([bench_func.get_concrete_function()])
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = gen_input_samples
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.uint8
    converter.inference_output_type = tf.uint8
    tflite_model = converter.convert()
    delegates = [load_delegate("libedgetpu.so.1.0")] if tpu else []
    interpreter = tf.lite.Interpreter(model_content=tflite_model, experimental_delegates=delegates)
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    tensor_in1 = interpreter.tensor(input_details[0]["index"])
    tensor_in2 = interpreter.tensor(input_details[1]["index"])
    tensor_out = interpreter.tensor(output_details[0]["index"])
    inp1 = np.random.rand(size, size) - 0.5
    inp2 = np.random.rand(size, size) - 0.5
    tensor_in1()[:] = inp1
    tensor_in2()[:] = inp2
    now = time()
    for _ in range(rep):
        interpreter.invoke()
    return time() - now
 
print(measure(2048, 1, tpu=False))

29.761183261871338
